In [2]:
import utils
from dataset import BuildingDataset
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from model import get_transform
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import torch

In [6]:
def add_gt_proposals(proposals, gt_boxes):
    # type: (List[Tensor], List[Tensor]) -> List[Tensor]
    proposals = [
        torch.cat((proposal, gt_box)) for proposal, gt_box in zip(proposals, gt_boxes)
    ]

    return proposals


proposals = [torch.rand((1, 4)) for _ in range(3)]
gt = [torch.rand((1, 4)) for _ in range(3)]

add_gt_proposals(proposals, gt)[0].shape

torch.Size([2, 4])

In [ ]:
# Load data for exploration.
dataset_test = BuildingDataset(
    "datasets/mlc_training_data/images_annotated/",
)

In [ ]:
# Load data for exploration.
dataset_expl = BuildingDataset(
    "datasets/mlc_training_data/images_annotated/",
)

data = [dataset_expl[i] for i in range(10)]
# data = [d for d in dataset_expl]

In [ ]:
img, masks, boxes = data[2][0], data[2][1]["masks"], data[2][1]["boxes"]

height_labels = [str(h.item()) for h in data[2][1]["building_heights"]]


def show_segmentation(img, masks, boxes=None, bcolors="red"):
    output_image = draw_segmentation_masks(img, masks.to(torch.bool), alpha=0.8)
    if boxes is not None:
        output_image = draw_bounding_boxes(
            output_image, boxes, height_labels, font_size=15, colors=bcolors
        )

    plt.figure(figsize=(12, 12))
    plt.imshow(output_image.permute(1, 2, 0))


show_segmentation(img, masks, boxes)

In [ ]:
height_labels = [d[1]["building_heights"] for d in data]
height_labels = torch.cat(height_labels).numpy()

counts, bins = np.histogram(
    height_labels,
    bins=[3, 6, 9, 15, 25, 40, 70, 120, 200, max(250, height_labels.max())],
)
plt.stairs(counts, bins, fill=True)

for q in [0.25, 0.5, 0.75, 0.95, 0.99]:
    print(f"Quantile {q}: {np.quantile(height_labels, q)}")

In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

dataset = BuildingDataset(
    "datasets/mlc_training_data/images_annotated/",
    transforms=get_transform(train=True),
)

data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn,
)

# For Training
images, targets = next(iter(data_loader))

images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)
print(predictions[0])

In [ ]:
from model import train

train(
    dataset_root="datasets/mlc_training_data/images_annotated",
    num_epochs=20,
    train_batch_size=10,
)

In [ ]:
from typing import Any


class Engine:
    name = "super shit"

    def __init__(self) -> None:
        self.year = "2020"

    def start(self):
        print("engines starts")

    def stop(self):
        print("engines stops")


class Car:
    def __init__(self, engine) -> None:
        self.engine = engine

    def __getattribute__(self, name: str) -> Any:
        if name == "forward":
            

        return getattr(super().__getattribute__("engine"), name)

    def stop(self):
        print(self.engine.year)
        print(self.engine.name)


engine = Engine()
car = Car(engine)

car.stop()
car.start()